# Amplitude Amplification Module

The present notebook reviews the *amplitude_amplification* module from package *AA* of the library *QQuantLib*  (**QQuantLib/AA/amplitude_amplification.py**).  All mandatory functions for creating Grover-like operators for **amplitude amplification** procedures are developed in this module.


The present notebook and module are based on the following references:

* *Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000). Quantum amplitude amplification and estimation.AMS Contemporary Mathematics Series, 305. https://arxiv.org/abs/quant-ph/0005055v1*
* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Data loading

Before doing any amplification we want to load some data into the quantum circuit, as this step is only auxiliary to see the effect of an amplification, we are just going to load a discrete probability distribution. In this case, we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$

Note that this probability distribution is properly normalised. 

For that purpose we will use the function *load_probability* from **QQuantLib/DL/data_loading**. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
p = x/np.sum(x)

In [ ]:
probability_routine = qlm.QRoutine()
register = probability_routine.new_wires(n)
probability_routine.apply(load_probability(p),register)
%qatdisplay probability_routine --svg

In [ ]:
results_loading, _, _, _ = get_results(probability_routine, linalg_qpu=linalg_qpu, shots=0)
amplitudes_loading = results_loading["Amplitude"].values
results_loading

In [ ]:
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_loading)
print('Test OK?: ',np.isclose(np.sqrt(p), amplitudes_loading).all())

See that the information stored in the quantum state is the same as the one stored in array $p$. For more information about loading data into the quantum circuit see the notebook *data_loading_use*.

## 2. Reflections

**Reflections** are the mandatory ingredients for amplification. A reflection rotates the phase of a state $\dfrac{\pi}{2}$. In other words, for real numbers **it changes the sign of the state**. 

The function *reflection* from **QQuantLib/AA/amplitude_amplification** module is used for creating **reflections**.

In [ ]:
from QQuantLib.AA.amplitude_amplification import reflection

Creating a reflection over a quantum state using the above-mentioned function is a two steps process:

1. **Creating the reflection QLM Abstract Gate**. This is done by calling the *reflection* function indicating the quantum state for doing the reflection. The state is a Python list and should provided when the user calls the function. **Ex: state = [1, 0, 0, 1] RG=reflection(state)**. The function will create a QLM AbstractGate.
2. Applying the resulting **reflection QLM Abstract Gate** over a QLM Program or QRoutine *specifying the qubits that will be affected by it*: **Ex: qlm_routine.apply(RG, [qbit[0], qbit[1], qbit[2],...])**

Let's give some examples.

**NOTE**
By default, the reflection function uses the default QLM construction for the mandatory multi-controlled Z gate. If we pass **mcz_qlm=False** a multiplexor version of the multi-controlled Z gate will be used.

### 2.1 First example

We will start with a very simple example, in this case, we want to flip the sign of state $|7\rangle$. For that, we will use the function *reflection*. This function takes a list as a parameter, the list specifies which combination of qubits is going to be reflected. In this case, the binary representation of the state $|7>$ is $111$, so the argument that we have to pass to the function is $[1,1,1]$.

In [ ]:
reflection1 = qlm.QRoutine()
reflection1_register = reflection1.new_wires(n)
reflection1.apply(probability_routine,reflection1_register)
#Step 1: Creating reflection QLM Abstract Gate
ReflectionGate = reflection([1,1,1])
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#ReflectionGate = reflection([1,1,1], mcz_qlm=False)
%qatdisplay ReflectionGate --depth  --svg
#Step 2: Applying ReflectionGate to the affected qbits: In present case the reflection gate affects all the qbits of the system
reflection1.apply(ReflectionGate, reflection1_register)
%qatdisplay reflection1 --depth 1 --svg

Now we print the amplitudes stored in the quantum circuit against the ones stored in the array $p$ to see the differences in sign.

In [ ]:
results_reflection1, _, _, _ = get_results(reflection1, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection1 = results_reflection1["Amplitude"].values
results_reflection1

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection1)

In [ ]:
print('Test OK: ',
    np.isclose(np.sqrt(p)[:-1], amplitudes_reflection1[:-1]).all() and
    np.isclose(np.sqrt(p)[-1], -amplitudes_reflection1[-1]).all()
     )

As we see the last state has changed from $0.5$ to $-0.5$.

### 2.2 Second example

In this example, we want to do something more difficult. We want to flip the sign of all states that start with one. Those states are $00\mathbf{1}\longrightarrow |1\rangle$, $01\mathbf{1}\longrightarrow |3\rangle$, $10\mathbf{1}\longrightarrow |5\rangle$ and $11\mathbf{1}\longrightarrow |7\rangle$. Again, we have to use the function *reflection*. This time we have to apply the reflection to the first qubit, so we just apply it to the first register.

In [ ]:
reflection2 = qlm.QRoutine()
reflection2_register = reflection2.new_wires(n)
reflection2.apply(probability_routine,reflection2_register)
#Step 1: Creating reflection QLM Abstract Gate
ReflectionGate2 = reflection([1])
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#ReflectionGate2 = reflection([1], mcz_qlm=False) 
%qatdisplay ReflectionGate2 --depth   --svg
#Step 2: Applying ReflectionGate to the affected qubits: In this case the first qubit
reflection2.apply(ReflectionGate2,reflection2_register[0])
%qatdisplay reflection2 --depth 1 --svg

Now we will see that the positions $1,3,5$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection2, _, _, _ = get_results(reflection2, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection2 = results_reflection2["Amplitude"].values
results_reflection2

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_reflection2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_reflection2[[1, 3, 5, 7]]
    ).all()
)

### 2.3 Third example

In this example, we will do a reflection in a more specific state. In this case, we want to do the reflection of the states $|1\rangle,|3\rangle,|5\rangle,|6\rangle,|7\rangle$. We can divide the process of the reflection into two sub-reflections.
- First, note that the reflection over states $|1\rangle,|3\rangle,|5\rangle,|7\rangle$ is the one implemented in Section 2.2.
- Second, the remaining state $|6\rangle$ can be reflected using the same strategy as in Section 2.1.

In [ ]:
reflection3 = qlm.QRoutine()
reflection3_register = reflection3.new_wires(n)
reflection3.apply(probability_routine,reflection3_register)
reflection3.apply(reflection([1]),reflection3_register[0])
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#reflection3.apply(reflection([1], mcz_qlm=False),reflection3_register[0])
reflection3.apply(reflection([1,1,0]),reflection3_register)
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#reflection3.apply(reflection([1,1,0], mcz_qlm=False),reflection3_register)
%qatdisplay reflection3 --depth 2 --svg

Now we will see that the positions $1,3,5,6$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection3, _, _, _ = get_results(reflection3, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection3 = results_reflection3["Amplitude"].values
results_reflection3

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection3)

In [ ]:
print('Test OK: ',
    np.isclose(
        amplitudes_reflection3[[1, 3, 5, 6, 7]],
        -np.sqrt(p)[[1, 3, 5, 6, 7]]
    ).all() and
    np.isclose(
        amplitudes_reflection3[[0, 2, 4]],
        np.sqrt(p)[[0, 2, 4]]
    ).all()
)

Note that doing consecutive reflections over sets is a commutative operation **when the sets are disjoint**.

## 3. Grover operator

In general, the Grover operator works in the following way. Let's say that we have an operator (a routine) that performs the following operation:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states. Now, the Grover operator $\mathcal{G}$ does the following transformation:
$$ |\Psi \rangle \longrightarrow \mathcal{G}^k|\Psi\rangle= \sin\left((2k+1)\theta\right)|\Psi_0\rangle +\cos\left((2k+1)\theta\right)|\Psi_1\rangle.$$ 
This operator is known in the literature as **amplitude amplification** as, in principle, it *increases* the probability of obtaining the state $|\phi\rangle$. Note that, when the angle $(2k+1)\theta$ goes over $\dfrac{\pi}{2}$, the probability instead of increase starts decreasing. Whenever we use the term amplification keep in mind that it can have this effect.

The Grover operator $\mathcal{G}$ can be decomposed in 2 different operators:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

In this section, we are going to review all the operators programmed for creating the Grover operator.



### 3.2 Operator $\hat{U}_{|\Psi_{0}\rangle}$

The first mandatory operator is:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

When we apply this operator on state $|\Psi\rangle$:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle = -\sin(\theta)|\Psi_{0}\rangle+\cos(\theta)|\Psi_{1}\rangle$$

So operator $\hat{U}_{|\Psi_{0}\rangle }$ is a reflection of the state $|\Psi_{0}\rangle$ around state $|\Psi_{1}\rangle$. Or in a graphic view:

![title](images/OraculeReflection.png)

The function *create_u0_gate* from **QQuantLib/AA/amplitude_amplification** module allows us to create an Abstract Gate for doing the operation. As in the case of the reflection, the application of the $\hat{U}_{|\Psi_{0}\rangle}$ to a quantum state will be a two steps process:

1. **Creating the $\hat{U}_{|\Psi_{0}\rangle}$ QLM AbstractGate** using the *create_u0_gate* and providing it, as input, the state that we want to flip the sign. This is done with the following three arguments:
    * The first one is always the oracle (this is the QLM QRoutine of the Program)
    * The second one is a list containing the binary representation of the state that we want to flip (marked state).
    * The third one is a list of the registers over which we want to act. 
2. **Applying $\hat{U}_{|\Psi_{0}\rangle}$ QLM AbstractGate** to the registers that will be affected.

In this sense, the syntax is very similar to that of the reflections. Indeed, it is implemented using the function *reflection*. Next, we give two examples.

**NOTE**
By default, the QLM construction for the multi-controlled Z gate is used. If we pass **mcz_qlm=False** a multiplexor version of the multi-controlled Z gate will be used.

In [ ]:
from QQuantLib.AA.amplitude_amplification import create_u0_gate

#### 3.2.1 First example

In this first example, we want to mark the state $|7\rangle$. Its binary representation is $111$, and we want to act on the register [0,1,2]. So following the above-mentioned arguments we call the *create_u0_gate* with the following arguments:
* First Input: Oracle that will be the QLM Routine with the loaded probability
* Second input: state we want to mark in binary representation: [1,1,1] 
* Third input: Registers over which we want to act: [0,1,2].

In [ ]:
routine_U0_1 = qlm.QRoutine()
register_U0_1 = routine_U0_1.new_wires(probability_routine.arity)
routine_U0_1.apply(probability_routine,register_U0_1)
#Creating U0_gate
U0_gate = create_u0_gate(
    probability_routine, #oracle
    [1,1,1], #marked state
    [0,1,2] #affected qbits
)
#Comment before 5 lines and uncoment following for For multiplexor implementation of multi-controlled Z gate
#U0_gate = create_u0_gate(probability_routine, [1,1,1], [0,1,2], mcz_qlm=False)
%qatdisplay U0_gate --depth  --svg
#Apply U0_gate
routine_U0_1.apply(U0_gate,register_U0_1)
%qatdisplay routine_U0_1 --depth 2 --svg

If we display the circuit with one more layer of depth, we will see that it is implemented as a reflection([1,1,1]) acting on the register [0,1,2].

Last, we will show the amplitudes stored in the quantum circuit and we will see that only the last element is affected

In [ ]:
results_U0_1,_ ,_ ,_  = get_results(routine_U0_1,linalg_qpu)

In [ ]:
amplitudes_U0_1 = results_U0_1["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_1)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[:-1],
        amplitudes_U0_1[:-1]
    ).all() and
    np.isclose(
        np.sqrt(p)[-1],
        -amplitudes_U0_1[-1]
    ).all()
)

#### 3.2.1 Second example

For the second example we want flip the sign/mark all states that start with one. Those states are $00\mathbf{1}\longrightarrow |1\rangle$, $01\mathbf{1}\longrightarrow |3\rangle$, $10\mathbf{1}\longrightarrow |5\rangle$ and $11\mathbf{1}\longrightarrow |7\rangle$. Again, we have to use the function *U0*, but this time we indicate that we are only acting upon the first register. So the arguments when calling *U0*  function will be:

* First Input: Oracle that will be the QLM Routine with the loaded probability
* Second input: state we want to mark in binary representation: [1] 
* Third input: Registers over which we want to act: [0].

In [ ]:
routine_U0_2 = qlm.QRoutine()
register_U0_2 = routine_U0_2.new_wires(probability_routine.arity)
routine_U0_2.apply(probability_routine,register_U0_1)
#Creating U0_gate
U0_2_gate = create_u0_gate(
    probability_routine, #oracle
    [1], #marked state
    [0] #affected qbits
)
#Comment before 5 lines and uncomment following for For multiplexor implementation of multi-controlled Z gate
#U0_2_gate = create_u0_gate(probability_routine, [1], [0], mcz_qlm=False) 
%qatdisplay U0_2_gate --depth 0 --svg
routine_U0_2.apply(U0_2_gate,register_U0_2)
#Apply U0_gate
%qatdisplay routine_U0_2 --depth 2 --svg

If we display the circuit with one more layer of depth, we will see that it is implemented as a *reflection([1])* acting on the register *[0]*.

Last, we will show the amplitudes stored in the quantum circuit and we will see that only states $|1\rangle$, $|3\rangle$, $|5\rangle$ and $|7\rangle$ are affected.

In [ ]:
results_U0_2,_ ,_ ,_ = get_results(routine_U0_2,linalg_qpu)

In [ ]:
amplitudes_U0_2= results_U0_2["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_U0_2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_U0_2[[1, 3, 5, 7]]
    ).all()
)

**Note that we cannot use operator *U0* to mark states that require more than one reflection. This is the case for example of the set of states $|1\rangle,|3\rangle,|5\rangle,|6\rangle,|7\rangle$.**

 ### 3.2 Operator $\hat{U}_{|\Psi}\rangle$

The $\hat{U}_{|\Psi\rangle}$ (**diffusor**):
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|,$$
is a reflection of state $|\Psi\rangle$ around the state $|\Psi\rangle^{\perp}$ (where $|\Psi\rangle^{\perp} \perp |\Psi\rangle$).

Using the graphic representation:

![title](images/StateReflection.png)

So:

$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = \hat{I}|\Psi\rangle - 2|\Psi\rangle \langle \Psi|\Psi\rangle = |\Psi\rangle - 2|\Psi\rangle = -|\Psi\rangle$$


For implementing this operator the function *create_u_gate* from **QQuantLib/AA/amplitude_amplification** module will be used. It only needs as input the oracle

**NOTE**
By default, the QLM construction for the multi-controlled Z gate is used. If we pass **mcz_qlm=False** a multiplexor version of the multi-controlled Z gate will be used.

In [ ]:
from QQuantLib.AA.amplitude_amplification import create_u_gate

In [ ]:
routine_U = qlm.QRoutine()
register_U = routine_U.new_wires(probability_routine.arity)
routine_U.apply(probability_routine,register_U)
routine_U.apply(create_u_gate(probability_routine),register_U) 
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#routine_U.apply(create_u_gate(probability_routine, mcz_qlm=False),register_U)
%qatdisplay routine_U --svg --depth 2

When we show the results we will see that all the amplitudes have changed their sign.

In [ ]:
results_U,_ ,_ ,_ = get_results(routine_U,linalg_qpu)

In [ ]:
amplitudes_U = results_U["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U)

In [ ]:
print('Test OK: ', np.isclose(np.sqrt(p), -amplitudes_U).all())

### 3.3 Grover

Now we have all the ingredients to create a Grover operator:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

The Grover operator can be implemented by function *grover* from **QQuantLib/AA/amplitude_amplification**. Let's see some examples

**NOTE**
By default, the QLM construction for the multi-controlled Z gate is used. If we pass **mcz_qlm=False** a multiplexor version of the multi-controlled Z gate will be used.

In [ ]:
from QQuantLib.AA.amplitude_amplification import grover

As in the case of the $\hat{U}_{|\Psi_{0}\rangle}$ operator, the application of the Grover operator to a quantum state will be a two steps process:

1. **Creating the Grover QLM AbstractGate** using the *grover* and providing it, as input, the state that we want to amplify. This is done with the following three arguments:
    * The first one is always the oracle (this is the QLM QRoutine of the Program)
    * The second one is a list containing the binary representation of the state that we want to amplify (marked state).
    * The third one is a list of the registers over which we want to act. 
2. **Applying Grover QLM AbstractGate** to the registers that will be affected.



### 3.3.1 First example

In our first example we are going to amplify the probability of obtaining the state $|1\rangle$.

In [ ]:
grover1 = qlm.QRoutine()
grover1_register = grover1.new_wires(n)
grover1.apply(probability_routine,grover1_register)
#Creating Grover_Gate_gate
Grover_Gate = grover(
    probability_routine, #oracle
    [0,0,1], #marked state
    [0,1,2] #affected qubits
)
#Comment before 5 lines and uncomment following for For multiplexor implementation of multi-controlled Z gate
#Grover_Gate = grover(probability_routine, [0,0,1], [0,1,2], mcz_qlm=False)
%qatdisplay Grover_Gate --depth 2 --svg
#Applying the Grover Gate
grover1.apply(Grover_Gate, grover1_register)
%qatdisplay grover1 --depth 3 --svg

The critical part is *grover(probability_routine,[0,0,1],[0,1,2])*.
The first argument of the function is always the routine that we are going to amplify. 
In the second argument, we have to specify our target state for amplification. In this case, as we want to amplify the state $|1\rangle$ we use its binary representation $001$.
Last, the third input is an index which specifies over which part of the quantum register we want to act. In this case, as the three quantum registers are involved, we put the three of them $[0,1,2]$. In the next examples, it will become clearer how it works.

Now we measure the amplified probabilities

In [ ]:
results_grover1,_ ,_ ,_ = get_results(grover1,linalg_qpu)
probabilities_grover1 = results_grover1["Probability"].values
probabilities_grover1

Let's explain what we have done so far. In our example, we do the following identifications:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \cos(\theta)|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
When we do the amplification and measure the probabilities we will obtain in position $1$ the new probability $p_{a} = \sin^2(3\theta)$. The original probability was $p_o = \sin^2(\theta)$. To check if the amplified probability and the original probability are correctly computed, we simply use the fact that, we can recover $\theta$ from the amplified probability by doing the operation:
$$ \theta = \dfrac{\arcsin\left(\sqrt{p_a}\right)}{3}.$$
Then we relate it with the original one substituting the angle, hence:
$$
p_o = \sin ^2\left(\dfrac{\arcsin\left(\sqrt{p_a}\right)}{3}\right)
$$
In the next cell, we check that the amplification is done correctly

In [ ]:
unamplified_grover1 = np.sin(np.arcsin(np.sqrt(probabilities_grover1[1]))/3)**2
print("Original probabilities: ", p[1])
print("Quantum probabilities: ",unamplified_grover1)

### 3.2 Second example

In this second example we are going to show how to amplify a more complex state. Say that now we want to amplify the state composed by $|0\rangle,|1\rangle,|2\rangle,|3\rangle$. Now we will see the correspondence of this new setup with the Grover setup:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\phi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle\right].\\
    & \cos(\theta)|\phi^\dagger\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
In binary representation, the states $|0\rangle,|2\rangle,|4\rangle,|6\rangle$ are $|\mathbf{0}00\rangle,|\mathbf{0}01\rangle,|\mathbf{0}10\rangle,|\mathbf{0}11\rangle$ respectively. Here we are interested in the join probability of getting these states, this is the same as asking for the probability of the leftmost qubit being zero.
Before doing an amplification we are going to do the unamplified version.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
%qatdisplay grover2 --depth 0 --svg

In [ ]:
results_grover2,_,_,_  = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

When we only measure the leftmost qubit ($q_2$) we are effectively performing the following operation:
$$ 
    \begin{array}{l}
p_{|0\rangle} = \sin^2\left(\theta\right) = \dfrac{1}{0+1+2+3+4+5+6+7+8} \left[|\sqrt{0}|^2+|\sqrt{1}|^2+|\sqrt{2}|^2+|\sqrt{3}|^2\right].\\
p_{|1\rangle} = \cos^2\left(\theta\right) = \dfrac{1}{0+1+2+3+4+5+6+7+8} \left[|\sqrt{4}|^2+|\sqrt{5}|^2+|\sqrt{6}|^2+|\sqrt{7}|^2\right].\\
\end{array}
$$

In the next cell, we check it with the classical probability

In [ ]:
print(" Probability of state |0>: ", p[0]+p[1]+p[2]+p[3])
print(" Probability of state |1>: ", p[4]+p[5]+p[6]+p[7])

Next, we will amplify the probability of the state marked with the rightmost qubit being $0$. For that, we again use the *grover* function. Apart from the oracle we need to indicate the state that we want to amplify.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
#Creating Grover_Gate_gate
Grover_Gate_2 = grover(
    probability_routine, #oracle
    [0], #marked state
    [n-1] #affected qbits
)
#Comment before 5 lines and uncoment following for For multiplexor implementation of multi-controlled Z gate
Grover_Gate_2 = grover(probability_routine, [0], [n-1], mcz_qlm=False)
%qatdisplay Grover_Gate_2 --depth 3 --svg
#Applying the Grover Gate
grover2.apply(Grover_Gate_2,grover2_register)
%qatdisplay grover2 --svg

Now we will measure again the probabilities of the leftmost qubit ($q_2$) being zero.

In [ ]:
results_grover2,_,_,_ = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

We clearly see that we have increased the probability of obtaining $|0\rangle$ at the cost of decreasing the probability of obtaining $|1\rangle$. Again we can see the correspondence of this amplified probability with the unamplified one.

In [ ]:
unamplified_grover2 = np.sin(np.arcsin(\
                    np.sqrt(results_grover2["Probability"].iloc[0]))/3)**2
print("Unamplified probability of state |0>: ", unamplified_grover2)
print("Classical probability of state |0>: ", p[0]+p[1]+p[2]+p[3])


### NOTE

In the module *utils* of package *utils* of the library *QQuantLib*  (**QQuantLib/utils/utils.py**) we have developed the *load_qn_gate* function that allows the use to create several applications of one given gate. We can use it for doing a multiple application of a Grover-like operator:

In [ ]:
from QQuantLib.utils.utils import load_qn_gate

In [ ]:
n_grover = load_qn_gate(grover(probability_routine,[0],[n-1]), 5)

In [ ]:
%qatdisplay n_grover --depth 1 --svg